# Eigene Tweets explorieren

## Vorbereitung

Mit diesem Notebook können Sie Ihre eigenen Tweets explorieren. Um dieses mit ihren eigenen Daten nutzen zu können, müssen die zuvor Ihr [Twitter-Archiv herunterladen](https://help.twitter.com/en/managing-your-account/how-to-download-your-twitter-archive). Nachdem Sie Ihr Archiv gespeichert haben, müssen Sie die `.zip`-Datei zunächst entpacken.

**Hinweis**: Ihr Twitter-Archiv enthält mehr Daten als nur Ihre Tweets. Wenn Sie die Datei entpackt haben, können Sie Ihre Daten auch über ihren Browser explorieren. Hierzu müssen Sie auf Ihrem Rechner lokal die Datei `Your archive.html` öffnen.

Die Daten liegen in Ihrem Twitter-Archiv-Ordner im Unterordner `data` im [JSON-Format](https://de.wikipedia.org/wiki/JavaScript_Object_Notation) vor. JSON-Dateien haben eigentlich die Endung `.json`. Twitter nutzt für die Exportdateien jedoch eine nicht-standardisierte Version von JSON-Listen mit der Dateiendung `.js` (die eigentlich für JavaScript-Dateien genutzt wird). Um die Daten in `R` einlesen zu können, müssen die Dateien zuerst manuell verändert werden.

Für dieses Notebook nutzen wir die Daten zu den Tweets aus der Datei `tweet.js`. Um diese mit `R` einlesen und explorieren zu können, müssen Sie zwei Dinge (lokal auf Ihrem Rechner) machen:
1. Die Datei `tweet.js` mit einem Texteditor wie [Notepad++](https://notepad-plus-plus.org/) oder [Atom](https://atom.io/) öffnen und den Text vor der ersten eckigen Klammer löschen (in meiner Datei war dies der folgende Text: "window.YTD.tweet.part0 = ").
1. Die Datei mit der Endung `.json` speichern (z.B. als `tweet.json`).

Nachdem Sie dies gemacht haben, müssen Sie die Datei hier noch hochladen, um damit arbeiten zu können. Die Datei sollte im Ordner *data* gespeichert werden. Öffnen Sie diesen (durch Doppelklick auf den Ordnernamen) im File Explorer auf der linken Seite und nutzen dann den *Upload Files*-Button im Menü oben links (das Symbol ist ein Pfeil über einem Strich). Wählen Sie darüber die JSON-Datei, die Sie gerade erstellt haben, von Ihrem lokalen Rechner aus und laden Sie diese in den Ordner *data* hoch.

**Hinweis**: Wenn Sie Twitter schon sehr lange nutzen und häufig tweeten, wird die Datei, die Ihre Tweets enthält, recht groß sein. In diesem Fall können der Upload sowie das Einlesen (s.u.) etwas länger dauern (dies gilt ggf. auch für das lokale Öffnen und Bearbeiten der Datei).

**Zur Erinnerung**: Ihre persönliche Kopie des Notebooks sowie alle Dateien, die Sie hochladen, werden am Ende der Nutzungssitzung gelöscht. Wenn Sie aber ganz "auf Nummer sicher gehen" wollen, können Sie die Datei mit Ihren Tweets auch über den File Explorer auf der linken Seite nach dem Durcharbeiten dieses Notebooks auch manuell löschen: Rechtsklick auf den Dateinamen und dann *Delete* auswählen.

## Pakete laden

Wie auch in den anderen Notebooks müssen wir zunächst die benötigten `R`-Pakete laden. Das sind diesmal ein paar mehr, da wir mit Textdaten und Zeitstempeln arbeiten und diese für unsere Analysen und Visualisierungen entsprechend aufbereiten müssen. Bevor wir die Pakete laden können, müssen wir noch das Paket `stopwords` (nach-)installieren, um später im Notebook auch mit deutschsprachigen Tweets arbeiten zu können (NB: die Installation dieses Pakets kann ein paar Minuten dauern).

In [ ]:
install.packages('stopwords')

In [ ]:
library(jsonlite)
library(tidyverse)
library(lubridate)
library(scales)
library(tidytext)
library(stopwords)
library(wordcloud)

## Daten einlesen

Im nächsten Schritt lesen wir die Daten ein und wandeln sie in ein `data.frame`-Objekt um, mit dem wir in `R` arbeiten können.

In [ ]:
tweets_json <- fromJSON("./data/tweet.json") 
tweets_df <- jsonlite::flatten(tweets_json)

Um einen ersten Eindruck von den Daten zu bekommen, können wir uns die Namen der enthaltenden Variablen sowie die ersten 10 Zeilen im Datensatz anschauen.

In [ ]:
names(tweets_df)
head(tweets_df, 10)

Hinweis: In der Dokumentation seiner API bietet Twitter u.a. auch eine [Beschreibung der Datenstruktur von JSON-Files für Tweets an](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet), aus der ersichtlich wird, welche Daten diese enthalten. Diese Beschreibung ist im Prinzip das Analogon zu einem Codebuch für Befragungsdaten.

## Daten aufbereiten

Um die Daten auswerten und visualisieren zu können, müssen sie in ein entsprechendes Format gebracht werden. Hierzu passen wir als ersten Schritt die Zeitstempel für die Tweets an.

In [ ]:
tweets_df <- tweets_df %>% 
  mutate(timestamp = parse_date_time(tweet.created_at, 
                                     orders = "abdHMSzY")) %>% 
  mutate(timestamp = with_tz(timestamp, tzone = "Europe/Berlin"))

### Wann haben Sie Ihren ersten Tweet geposted?

In [ ]:
min(tweets_df$timestamp)

## Anzahl von Tweets über die Zeit

Nun visualisieren wir die Anzahl von Tweets pro Monat.

In [ ]:
tweets_df %>%
  mutate(time_floor = floor_date(timestamp, unit = "1 month")) %>%
  count(time_floor) %>%
  ggplot(aes(x = as.Date(time_floor), y = n)) +
  geom_bar(stat = "identity") +
  scale_x_date(labels = date_format("%m-%Y"),
               date_breaks = "1 year",
               date_minor_breaks = "1 month",
               expand = expansion(mult=c(0,0))) +
  scale_y_continuous(expand = expansion(mult=c(0,0.1))) +
  labs(title = "Anzahl von Tweets pro Monat",
       x = "Monat",
       y = "Tweets")

## Wann tweeten Sie?

An welchen Wochentagen tweeten Sie besonders häufig?

In [ ]:
tweets_df %>% 
  ggplot(aes(x = wday(timestamp, label = TRUE, week_start = 1), y = stat(count))) +
  geom_bar() +
  theme(legend.position = "none") +
  xlab("Day of the Week") + ylab("Number of tweets")

Zu welchen Uhrzeiten tweeten Sie am meisten?

In [ ]:
tweets_df %>% 
  ggplot(aes(x = hour(timestamp), y = stat(count))) +
  geom_bar() +
  theme(legend.position = "none") +
  xlab("Hour of the Day") + ylab("Number of tweets")

## Beliebteste Tweets

Eine weitere Sache, die wir uns anschauen können, ist, welche Tweets am populärsten waren. Hierzu nutzen wir den Favorite Count. Alternativ könnten wir auch den Retweet Count nutzen (die entsprechende Variable heißt `tweet.retweet_count`).

In [ ]:
tweets_df %>% 
  arrange(desc(tweet.favorite_count)) %>% 
  select(tweet.id, tweet.full_text, tweet.favorite_count) %>% 
  head(10)

## Replies, Hashtags und Retweets

Wie viel Prozent Ihrer Tweets sind Replies?

In [ ]:
round(sum(!is.na(tweets_df$tweet.in_reply_to_status_id))/nrow(tweets_df)*100, 2)

Welcher Anteil Ihrer Tweets sind Retweets?

In [ ]:
tweets_df %>% 
  mutate(is_retweet = ifelse(str_detect(tweet.full_text, "^RT"), 1, 0)) %>% 
  count(is_retweet) %>% 
  mutate(percentage = round(n/nrow(tweets_df)*100, 2))

Wie viele Ihrer Tweets enthalten Hashtags?

In [ ]:
tweets_df %>% 
  mutate(has_hashtag = ifelse(str_detect(tweet.full_text, "#"), 1, 0)) %>% 
  count(has_hashtag) %>% 
  mutate(percentage = round(n/nrow(tweets_df)*100, 2))

## Worthäufigkeiten

Um zu explorieren, welche Wörter Sie am häufigsten nutzen, müssen wir die Daten noch weiter aufbereiten. Hierzu müssen wir Verfahren aus dem sogenannten [Text Mining](https://de.wikipedia.org/wiki/Text_Mining) einsetzen. Eine Einführung in das Text Mining ist im Rahmen dieses Notebooks (sowie auch dieses Seminars) nicht möglich. Im Netz gibt es jedoch zahlreiche gute Einführungen, wie z.B.:
- [Text Mining with R - A Tidy Approach](https://www.tidytextmining.com/) von Julia Silge und David Robinson
- [Text mining in R for the social sciences and digital humanities](https://tm4ss.github.io/docs/) von Andreas Niekler und Gregor Wiedemann

Eine sehr ausführliche Einführung in die Arbeit mit Textdaten bietet auch die Webseite [Automatisierte Inhaltsanalyse mit R](http://inhaltsanalyse-mit-r.de/index.html) von Cornelius Puschmann.

Wir nutzen im Folgenden das Paket [`tidytext`](https://github.com/juliasilge/tidytext) und der Code basiert in weiten Teilen auf den Beispielen aus Kapitel 7 des Buchs *Tidy Text Mining with R*. Ein anderes, ebenfalls sehr umfangreiches Paket für die Analyse von Textdaten in `R` ist [`quanteda`](https://quanteda.io/), zu dem es im Netz auch sehr umfangreiche [Tutorials](https://tutorials.quanteda.io/) gibt.

Im folgenden Aufbereitungsschritt teilen wir die Tweets in einzelne Wörter auf. Dies nennt man [Tokenization](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization). Zudem entfernen wir einige bestimmte Zeichenfolgen sowie sogenannte [Stoppwörter](https://de.wikipedia.org/wiki/Stoppwort) (d.h., Wörter, die sehr häufig in Texten vorkommen, für deren Inhalt jedoch i.d.R. nicht besonders wichtig sind) und schließen Retweets aus. Hierfür nutzen wir sogenannte [Regular Expressions](https://en.wikipedia.org/wiki/Regular_expression).

Ich selbst tweete auf Deutsch sowie auf Englisch. Da Stoppwörter sprachspezifisch sind, beschränken wir die Auswahl der Tweets im nachfolgenden Code auf solche, die Twitter als englischsprachig erkannt hat (Hinweis: Dies funktioniert natürlich nicht 100%ig; z.B. bei Tweets, die in mehreren Sprachen verfasst sind).

In [ ]:
remove_reg <- "&amp;|&lt;|&gt;"

tidy_tweets <- tweets_df %>% 
  rename(text = tweet.full_text) %>% 
  filter(!str_detect(text, "^RT"),
         tweet.lang == "en") %>%
  mutate(text = str_remove_all(text, remove_reg)) %>%
  unnest_tokens(word, text, token = "tweets") %>%
  filter(!word %in% stop_words$word,
         !word %in% str_remove_all(stop_words$word, "'"),
         str_detect(word, "[a-z]"))

Nun können wir Worthäufigkeiten zählen. Hierbei schließen wir @-mentions aus.

In [ ]:
word_freqs <- tidy_tweets %>% 
  filter(!str_detect(word, "^@")) %>% 
  count(word) %>% 
  rename(freq = n)

### Was sind die häufigsten Wörter in Ihren englischsprachigen Tweets?

In [ ]:
tidy_tweets %>% 
  filter(!str_detect(word, "^@")) %>% 
  count(word, sort = T) %>% 
  top_n(10) %>% 
  mutate(word = reorder(word, n)) %>% 
  ggplot(aes(x = word, y = n)) +
  geom_col() +
  labs(x = NULL, y = "Häufigkeit") +
  coord_flip()

Selbstverständlich können wir dasselbe auch für deutschsprachige Tweets machen. Hierfür müssen wir zusätzlich die entsprechende Liste mit Stoppwörtern laden.

### Was sind die häufigsten Wörter in Ihren deutschsprachigen Tweets?

In [ ]:
stop_words_de <- get_stopwords(language = "de")

saubere_zwitscher <- tweets_df %>% 
  rename(text = tweet.full_text) %>% 
  filter(!str_detect(text, "^RT"),
         tweet.lang == "de") %>%
  mutate(text = str_remove_all(text, remove_reg)) %>%
  unnest_tokens(word, text, token = "tweets") %>%
  filter(!word %in% stop_words_de$word,
         str_detect(word, "[a-z]"))

wrt_hfgkt <- saubere_zwitscher %>% 
  filter(!str_detect(word, "^@")) %>% 
  count(word) %>% 
  rename(freq = n)

wrt_hfgkt %>% 
  arrange(-freq) %>% 
  head(10)

### Vergleich der häufigsten deutschen und englischen Wörter in Ihren Tweets

Nachfolgend kombinieren wir die englischsprachigen und deutschsprachigen Tweets und vergleichen visuell die jeweiligen Worthäufigkeiten.

In [ ]:
tweets_en <- tidy_tweets %>% 
  select(word, tweet.lang) %>% 
  rename(language = tweet.lang)

tweets_de <- saubere_zwitscher %>% 
  select(word, tweet.lang) %>% 
  rename(language = tweet.lang)

tweets_combined <- tweets_en %>% 
  bind_rows(tweets_de)

tweets_combined %>% 
  filter(!str_detect(word, "^@"),
         !str_detect(word, "^#")) %>% 
  count(word, language, sort = T) %>% 
  ungroup %>% 
  group_by(language) %>%
  top_n(10, n) %>%
  ungroup() %>%
  mutate(word = reorder(word, n, fill = language)) %>% 
  ggplot(aes(x = word, y = n)) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~language, scales = "free_y") +
  labs(x = NULL, y = "Häufigkeit") +
  coord_flip()

### Häufigste Hashtags

Welche Hashtags verwenden Sie am häufigsten?

In [ ]:
tweets_combined %>% 
  filter(str_detect(word, "^#")) %>% 
  count(word) %>%
  arrange(-n) %>% 
  head(10)

## Wortwolken

Eine populäre Form der Visualisierung von Textdaten sind Wortwolken (word clouds). Auch diese lassen sich mit `R` einfach erstellen (*NB*: wie so oft in `R`, gibt es auch hierfür verschiedene Pakete).

In [ ]:
wordcloud(words = word_freqs$word,
          freq = word_freqs$freq,
          min.freq = 10,
          max.words = 50,
          random.order = FALSE,
          rot.per = 0.35,
          colors = brewer.pal(8, "Dark2"))

wordcloud(words = wrt_hfgkt$word,
          freq = wrt_hfgkt$freq,
          min.freq = 5,
          max.words = 50,
          random.order = FALSE,
          rot.per = 0.35,
          colors = brewer.pal(8, "Dark2"))

## Ausblick

Noch umfangreichere Daten über die eigenen Twitter-Aktivitäten sowie die anderer Nutzer\_innen kann man über die [Twitter API](https://developer.twitter.com/en/docs/twitter-api) sammeln. In `R` geht dies am einfachsten und umfassendsten mit dem Paket [`rtweet`](https://docs.ropensci.org/rtweet/). Zu diesem gibt es im Netz verschiedene Tutorials, wie z.B. die [*21 Recipes for Mining Twitter Data with rtweet*](https://rud.is/books/21-recipes/) von Bob Rudis.

Seit einiger Zeit gibt es eine [neue Version der Twitter API (v2)](https://developer.twitter.com/en/docs/twitter-api). Für diese gibt es auch eine [eigene Zugangsoption für die akademische Forschung](https://developer.twitter.com/en/products/twitter-api/academic-research), die noch umfangreicheren Datenzugriff erlaubt als die kostenfreie Version der Standard-API v1.1. Auch für diese Zugriffsoption gibt es diverse R-Pakete. Das umfangreichste ist [academictwitteR](https://github.com/cjbarrie/academictwitteR).